## 1. セットアップ

## 1. 環境確認

In [ ]:
# GPU確認
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")

## 2. パッケージインストール

In [ ]:
# 必要パッケージのインストール
!pip install -q -r ../requirements.txt

## 3. モジュールインポート

In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import torch

# プロジェクトルートをパスに追加
sys.path.append('/workspace/atma_22_ca/')

from configs.config import *
from src.runner import Runner
from src.model_arcface import ModelArcFace
from src.util import Logger, Submission

print(f"Input dir: {DIR_INPUT}")
print(f"Model dir: {DIR_MODEL}")
print(f"Submission dir: {DIR_SUBMISSIONS}")

## 4. ロガー設定

In [ ]:
logger = Logger(path=DIR_LOG)

def get_run_name(model_type):
    run_name = model_type
    suffix = '_' + datetime.now().strftime("%Y%m%d%H%M")
    run_name = run_name + suffix
    return run_name

logger.info("Logger initialized")

## 5. データ読み込み

In [ ]:
df_train = pd.read_csv(FILE_TRAIN_META)
df_test = pd.read_csv(FILE_TEST_META)

# グループ化
df_train['group'] = df_train['quarter'] + '_' + df_train['session'].astype(str)

print(f"Train: {df_train.shape}")
print(f"Test: {df_test.shape}")
print(f"\n【CV Strategy】")
print(f"  Group column: 'group' (quarter_session)")
print(f"  Total groups: {df_train['group'].nunique()}")
print(f"  Avg samples per group: {len(df_train) / df_train['group'].nunique():.1f}")

## 6. パラメータ設定（ArcFace + GPU最適化）

### GPU最適化戦略
- batch_size: GPU VRAMに応じて動的調整
- num_workers: CPU並列化でデータローディング高速化
- ArcFace損失: 角度マージンでクラス間分離を強化

In [ ]:
memo = "EfficientNet-B0 + ArcFace"
run_name = get_run_name("arcface_efficientnet_b0")
print(f"Run name: {run_name}")
print(f"Memo: {memo}")

In [ ]:
# デバイス確認とGPU最適化パラメータ設定
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")

if device == 'cuda':
    # GPU仕様を確認
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3  # GB
    print(f"GPU: {gpu_name}, Memory: {gpu_memory:.1f}GB")
    
    # CPU並列化を強化
    import os
    cpu_count = os.cpu_count() or 8
    print(f"利用可能CPU: {cpu_count} cores")
    
    # 大容量GPU (20GB以上) の場合
    if gpu_memory > 20:
        batch_size = 128  # ArcFaceは少し控えめ（KNNより学習コスト高）
        num_workers = min(12, cpu_count)
        epochs = 20
    else:
        batch_size = 64
        num_workers = min(8, cpu_count)
        epochs = 20
else:
    batch_size = 32
    num_workers = 2
    epochs = 10  # CPU環境では短縮

print(f"最適化パラメータ: batch_size={batch_size}, num_workers={num_workers}, epochs={epochs}")

In [ ]:
# ArcFaceモデルパラメータ
params = {
    # モデル設定
    'model_name': 'efficientnet_b0',  # バックボーン: 'efficientnet_b0', 'resnet50', etc.
    'embedding_dim': 512,              # 埋め込みベクトル次元数
    'img_size': 224,                   # 入力画像サイズ
    
    # ArcFace固有パラメータ
    'arcface_s': 30.0,                 # スケールファクター（論文推奨値）
    'arcface_m': 0.5,                  # 角度マージン（0.5 rad ≈ 28.6度）
    
    # 訓練設定
    'batch_size': batch_size,          # GPU環境に応じて調整
    'epochs': epochs,                  # エポック数
    'lr': 1e-3,                        # 学習率
    'weight_decay': 1e-4,              # 重み減衰
    
    # EMA設定
    'use_ema': True,                   # EMA（Exponential Moving Average）を使用
    'ema_decay': 0.995,                # EMA減衰率
    
    # 推論設定
    'threshold': 0.5,                  # 未知選手判定の閾値
    
    # その他
    'num_workers': num_workers,        # DataLoaderのワーカー数
}

# CV設定
cv_setting = {
    'method': 'group',
    'group_col': 'group',
    'n_splits': 5,
}

print("\n=== Parameters ===")
for key, value in params.items():
    print(f"  {key}: {value}")

## 7. Runner作成

In [ ]:
# Runnerの作成
runner = Runner(
    run_name=run_name,
    model_cls=ModelArcFace,
    params=params,
    df_train=df_train,
    df_test=df_test,
    cv_setting=cv_setting,
    logger=logger
)

print(f"\nRunner created: {run_name}")
print(f"Model class: {ModelArcFace.__name__}")

## 8. 5-Fold CV学習（ArcFace訓練）

各Foldで以下を実行：
1. ArcFace損失で埋め込み学習（約20エポック）
2. EMAで重み平滑化
3. 訓練データからプロトタイプ（平均埋め込み）を計算
4. モデル・プロトタイプ保存

In [ ]:
%%time
# 5-fold CV学習
runner.train_cv()

## 9. 5-Fold評価

In [ ]:
scores, oof_score = runner.metric_cv()

## 10. テストデータ予測

In [ ]:
%%time
# CV全モデルのアンサンブル予測
pred_test = runner.predict_cv()

In [ ]:
# 予測分布の可視化
plt.figure(figsize=(10, 5))
pred_test['label_id'].value_counts().sort_index().plot(kind='bar')
plt.title('Test Prediction Distribution (ArcFace)')
plt.xlabel('Label ID')
plt.ylabel('Count')
plt.axhline(y=len(pred_test)/11, color='r', linestyle='--', alpha=0.5, label='Uniform')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 11. 提出ファイル作成

In [ ]:
Submission.save(pred_test, run_name=run_name, logger=logger)

## 12. 閾値チューニング実験（高速）

学習済みモデルを使って異なる閾値で再予測

In [ ]:
# 閾値を変えて予測（学習済みモデル再利用）
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]
threshold_results = []

print("\n=== 閾値チューニング ===")
print("(学習済みモデルを再利用して高速実験)\n")

for thresh in thresholds:
    # パラメータ更新
    params_test = params.copy()
    params_test['threshold'] = thresh
    
    # 新しいRunnerで予測のみ
    runner_test = Runner(
        run_name=run_name,  # 同じrun_nameでモデル読み込み
        model_cls=ModelArcFace,
        params=params_test,
        df_train=df_train,
        df_test=df_test,
        cv_setting=cv_setting,
        logger=logger
    )
    
    # 予測のみ実行
    pred = runner_test.predict_cv()
    
    # 統計情報
    unknown_count = (pred['label_id'] == -1).sum()
    unknown_ratio = unknown_count / len(pred) * 100
    
    threshold_results.append({
        'threshold': thresh,
        'unknown_count': unknown_count,
        'unknown_ratio': unknown_ratio
    })
    
    print(f"Threshold {thresh:.1f}: unknown={unknown_count:4d} ({unknown_ratio:5.2f}%)")

df_threshold_results = pd.DataFrame(threshold_results)
print("\n=== 閾値チューニング結果 ===")
print(df_threshold_results)

In [ ]:
# 閾値とunknown比率の関係を可視化
if len(threshold_results) > 0:
    plt.figure(figsize=(10, 5))
    plt.plot(df_threshold_results['threshold'], df_threshold_results['unknown_ratio'], marker='o', linewidth=2)
    plt.xlabel('Threshold', fontsize=12)
    plt.ylabel('Unknown Ratio (%)', fontsize=12)
    plt.title('ArcFace: Effect of Threshold on Unknown Predictions', fontsize=14)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
print("\n※ 閾値が低い → unknown予測が増える（保守的）")
print("※ 閾値が高い → unknown予測が減る（積極的）")

## 13. 実験サマリー

In [ ]:
print("="*60)
print("実験サマリー")
print("="*60)
print(f"Run name: {run_name}")
print(f"Model: {params['model_name']} + ArcFace")
print(f"Embedding dim: {params['embedding_dim']}")
print(f"ArcFace (s={params['arcface_s']}, m={params['arcface_m']})")
print(f"EMA: {params['use_ema']} (decay={params['ema_decay']})")
print(f"Epochs: {params['epochs']}, Batch size: {params['batch_size']}")
print(f"Threshold: {params['threshold']}")
print(f"Unknown predictions: {(pred_test['label_id'] == -1).sum()} / {len(pred_test)}")
print("="*60)

print("\n【改善案】")
print("1. 閾値チューニング: threshold を 0.3 ~ 0.7 で実験")
print("2. バックボーン変更: resnet50, efficientnet_b1 等を試す")
print("3. データ拡張強化: MixUp, CutMix 等の追加")
print("4. マルチビュー活用: top/side画像の統合")
print("5. 後処理: 時系列スムージング、選手交代タイミング検出")
print("6. アンサンブル: KNNモデルとの組み合わせ")

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import torch

# 自作モジュールの読み込み
sys.path.append(os.path.abspath('..'))
from configs.config import *
from src.runner import Runner
from src.model_arcface import ModelArcFace
from src.util import Logger, Util, Metric

print(f"Input dir: {DIR_INPUT}")
print(f"Model dir: {DIR_MODEL}")
print(f"Submission dir: {DIR_SUBMISSIONS}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 2. ロガーの設定

In [ ]:
# ロガーの初期化
logger = Logger(path=DIR_LOG)

def get_run_name(model_type):
    """run名の作成"""
    run_name = model_type
    suffix = '_' + datetime.now().strftime("%Y%m%d%H%M")
    run_name = run_name + suffix
    return run_name

logger.info("Logger initialized")

## 3. データ読み込み

In [ ]:
# メタデータ読み込み
df_train = pd.read_csv(FILE_TRAIN_META)
df_test = pd.read_csv(FILE_TEST_META)

print(f"Train: {df_train.shape}")
print(f"Test: {df_test.shape}")

# ✅ リークしないCV戦略: quarter_sessionでグループ化
df_train['group'] = df_train['quarter'] + '_' + df_train['session'].astype(str)

print(f"\n【CV Strategy】")
print(f"  Group column: 'group' (quarter_session)")
print(f"  Total groups: {df_train['group'].nunique()}")
print(f"  Avg samples per group: {len(df_train) / df_train['group'].nunique():.1f}")
print(f"\n  Top 5 groups by size:")
print(df_train['group'].value_counts().head())

# データの確認
print(f"\n【Data Overview】")
print(df_train.head())
print(f"\nLabel distribution:")
print(df_train['label_id'].value_counts().sort_index())

## 4. パラメータ設定（ArcFace）

In [ ]:
# メモ
memo = "EfficientNet-B0 + ArcFace (metric learning)"

# run_name
run_name = get_run_name(model_type="arcface_efficientnet_b0")

print(f"Run name: {run_name}")
print(f"Memo: {memo}")

In [ ]:
# ArcFaceモデルパラメータ
params = {
    # モデル設定
    'model_name': 'efficientnet_b0',  # バックボーン: 'efficientnet_b0', 'resnet50', etc.
    'embedding_dim': 512,              # 埋め込みベクトル次元数
    'img_size': 224,                   # 入力画像サイズ
    
    # ArcFace固有パラメータ
    'arcface_s': 30.0,                 # スケールファクター（論文推奨値）
    'arcface_m': 0.5,                  # 角度マージン（0.5 rad ≈ 28.6度）
    
    # 訓練設定
    'batch_size': 64,                  # バッチサイズ（GPU環境に応じて調整）
    'epochs': 20,                      # エポック数
    'lr': 1e-3,                        # 学習率
    'weight_decay': 1e-4,              # 重み減衰
    
    # EMA設定
    'use_ema': True,                   # EMA（Exponential Moving Average）を使用
    'ema_decay': 0.995,                # EMA減衰率（0.995 ~ 0.9998が一般的）
    
    # 推論設定
    'threshold': 0.5,                  # 未知選手判定の閾値（最大類似度 < threshold → -1）
    
    # その他
    'num_workers': 4,                  # DataLoaderのワーカー数
}

logger.info(f"Parameters: {params}")

## 5. CV設定

In [ ]:
# CV設定（quarter_sessionでグループ化してリーク防止）
cv_setting = {
    "method": "group",              # グループKFold
    "group_col": "group",           # グループ列名
    "n_splits": 5,                  # Fold数
    "shuffle": True,
    "random_state": 42
}

logger.info(f"CV setting: {cv_setting}")

## 6. Runnerの作成

In [ ]:
# Runnerの作成
runner = Runner(
    run_name=run_name,
    model_cls=ModelArcFace,
    params=params,
    df_train=df_train,
    df_test=df_test,
    cv_setting=cv_setting,
    logger=logger
)

logger.info("Runner created successfully")

## 7. CV訓練

各Foldで以下を実行：
1. ArcFace損失で埋め込み学習
2. EMAで重み平滑化
3. 訓練データからプロトタイプ（平均埋め込み）を計算
4. モデル・プロトタイプ保存

In [ ]:
# CV訓練実行
runner.train_cv()

logger.info("Training completed")

## 8. CV予測

各Foldのモデルで予測し、アンサンブル：
1. テスト画像から埋め込み抽出
2. プロトタイプとのコサイン類似度計算
3. 全Foldの類似度を平均してアンサンブル

In [ ]:
# CV予測実行
runner.predict_cv()

logger.info("Prediction completed")

## 9. 提出ファイル作成

In [ ]:
# 提出ファイル保存
submission_path = runner.save_submission()

print(f"\n✓ Submission saved: {submission_path}")

# 提出ファイルの確認
submission = pd.read_csv(submission_path, header=None, names=['label_id'])
print(f"\nSubmission shape: {submission.shape}")
print(f"\nLabel distribution:")
print(submission['label_id'].value_counts().sort_index())
print(f"\nUnknown count: {(submission['label_id'] == -1).sum()}")

## 10. 結果の可視化

In [ ]:
# 予測分布の可視化
plt.figure(figsize=(12, 5))

# 訓練データのラベル分布
plt.subplot(1, 2, 1)
df_train['label_id'].value_counts().sort_index().plot(kind='bar', color='skyblue')
plt.title('Training Label Distribution')
plt.xlabel('Label ID')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)

# 予測ラベル分布
plt.subplot(1, 2, 2)
submission['label_id'].value_counts().sort_index().plot(kind='bar', color='lightcoral')
plt.title('Prediction Label Distribution')
plt.xlabel('Label ID')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(DIR_FIGURES, f'{run_name}_label_distribution.png'), dpi=150, bbox_inches='tight')
plt.show()

logger.info("Visualization completed")

## 11. 実験メモ

In [ ]:
print("="*60)
print("実験サマリー")
print("="*60)
print(f"Run name: {run_name}")
print(f"Model: {params['model_name']}")
print(f"Embedding dim: {params['embedding_dim']}")
print(f"ArcFace (s={params['arcface_s']}, m={params['arcface_m']})")
print(f"EMA: {params['use_ema']} (decay={params['ema_decay']})")
print(f"Epochs: {params['epochs']}, Batch size: {params['batch_size']}")
print(f"Threshold: {params['threshold']}")
print(f"Unknown predictions: {(submission['label_id'] == -1).sum()} / {len(submission)}")
print(f"Submission: {submission_path}")
print("="*60)

# 次の実験のための改善案
print("\n【改善案】")
print("1. 閾値チューニング: threshold を 0.3 ~ 0.7 で実験")
print("2. バックボーン変更: resnet50, efficientnet_b1 等を試す")
print("3. データ拡張強化: MixUp, CutMix 等の追加")
print("4. マルチビュー活用: top/side画像の統合")
print("5. 後処理: 時系列スムージング、選手交代タイミング検出")

## 12. 閾値チューニング（オプション）

訓練済みモデルを使って、異なる閾値で高速実験

In [ ]:
# 閾値を変えて再予測（学習済みモデルを再利用）
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]

print("閾値別のunknown予測数:")
print("-" * 40)

for thresh in thresholds:
    # パラメータ更新
    params_test = params.copy()
    params_test['threshold'] = thresh
    
    # 新しいRunnerで予測のみ実行
    runner_test = Runner(
        run_name=run_name,  # 同じrun_nameでモデル読み込み
        model_cls=ModelArcFace,
        params=params_test,
        df_train=df_train,
        df_test=df_test,
        cv_setting=cv_setting,
        logger=logger
    )
    
    # 予測のみ（訓練はスキップ）
    runner_test.predict_cv()
    
    # 提出ファイル確認
    sub_path = runner_test.save_submission()
    sub = pd.read_csv(sub_path, header=None, names=['label_id'])
    unknown_count = (sub['label_id'] == -1).sum()
    unknown_ratio = 100.0 * unknown_count / len(sub)
    
    print(f"Threshold {thresh:.1f}: {unknown_count:4d} ({unknown_ratio:5.2f}%)")

print("-" * 40)
print("\n※ 閾値が低い → unknown予測が増える")
print("※ 閾値が高い → unknown予測が減る")